# JupyterNotebook de la App para Electrobuzz

## Intro

En este cuaderno se pretende ir añadiendo todos los avances relativos al proyecto del desarrollo de la App para la descarga de música mediante Electrobuzz-Cosmobox. Cabe mencionar que este proyecto no está pensado para realizar un web-scrapping ilícito. Lo que se pretende mediante este programa es agilizar el proceso de descarga de música de Electrobuzz, pero teniendo en cuenta que es necesario contar con una cuenta de Cosmobox para ello. Tambien se pretende aprender a programar un scraper en python y hacer una pequeña interfaz de usuario que pueda acabar en convertirse en un ejecutable, QUIERO APRENDER A HACER SOFTWARE, PERO SOFTWARE CACHARRO. 

## Fundamentos

En principio esta app se basará en realizar peticiones mediante el paquete requests. Se ha planteado realizar el trabajo inicial en 3 rondas de peticiones. 

En primera instancia todos las urls de todos los albumes disponibles en Electrobuzz. 
Una vez se consiga tener un listado de todos los links se pretende realizar una seguna ronda de requests que se encarge de completar la información de cada album, asi como Artista, Remixers, tamaño de la descarga y SOBRE TODO comprobar que exista el link de descarga de Cosmobox (existen albumes que no estan disponibles para descargar en Cosmobox y por lo tanto no son accesibles para nosotros). 
En la tercera ronda de requests se pretende comprobar que el link de Cosmobox no está caído (Hay  bastantes links que no funcionan).

Con toda esta informacion se pasará a crear una interfaz empleando el paquete tkinter, la cual debe contar con un motor de busqueda dentro de toda la información de electrobuzz que nos permita rápidamente seleccionar los albumes que se quieren descargar. Sería interensante que dentro de este motor de busqueda se pudiera filtrar por nombre, sello, fecha, etc, para agilizar la labor de buscar la mñusica que nos apetezca. 

Dentro de la interfaz de tkinter habrá un botón para actualizar la información contenida en el software, como un método para tener al día todos los realeases de la página... Aunque tambien se podría realizar una actualización de manera automática. 

Debe ser sencillo selecionar la música que queremos descargar, igual añadimos un click marker y un botón de descarga al final de la página. Ya veré. Este paso no se si se realizará con requests (espero que si) o con Selenium... ambos tienen sus ventajas y desventajas. 


A continuación voy a añadir las librerias que voy empleando. Destacando requests y BeautifulSoup. 






In [42]:
from bs4 import BeautifulSoup
import requests

import shutil
import numpy as np
import os
import random

## Problemas detectados

El primero y yo creo que más importante es que los cabrones de Electrobuzz no dejan realizar scraping... Esto se puede confirmar viendo el archivo robot.txt de la página web. 

![Robot.txt de la página Electrobuzz](img_jupyter/robot.png)

El asterísco en User-Agent siginifica que no se permiten spyders, crawlers y compañia. Por lo tanto intentarán bloquear cualquier comportamiento que sea similar al de un boot, por ejemplo, muchas requests en poco tiempo, o muchas requests iguales. Una vez han detectado a un boot lo siguiente que hacen es bloquear la IP que está realizando las peticiones. La IP la marca tu router, pa entendernos, por lo tanto hay que usar VPN o proxies. Investigando un poquito llegue a la conclusión de que iba a ser más fácil incluir el método de cambiar de proxies, sobre todo porque el paquete requests ya incluye métodos para fijar una proxie. 

## Buscar y cambiar proxy

Para esto es necesario saber que las proxies no son fijas, es decir, las proxies son puntos de conexion que no siempre se encuentran disponibles por ello es necesario ir actualizando la lista de proxies a las que vamos a conectarnos. Para ello yo voy a usar una página web que ofrece un servicio de proxies gratuitas. 

Creo una función que devuelve las proxies encontradas en la página web. 

In [43]:
def get_proxies():   
   
  proxies_req = requests.get('https://www.sslproxies.org/')


  soup = BeautifulSoup(proxies_req.content, 'html.parser')
  proxies_table = soup.find(id='proxylisttable')
  
  proxies=[]

  # Save proxies in the array
  for row in proxies_table.tbody.find_all('tr'):
    proxies.append({
      'ip':   row.find_all('td')[0].string,
      'port': row.find_all('td')[1].string
    })

  return proxies



get_proxies() [:3]

[{'ip': u'186.225.97.246', 'port': u'43082'},
 {'ip': u'170.238.41.16', 'port': u'31208'},
 {'ip': u'125.25.165.97', 'port': u'47106'}]

Es necesario entender como se deben incluir las proxies a las requests. Sin más, te lo demuestro con la función que he creado para poner en formato las proxies par que las entienda el paquete request. 

A esta función hay que meter como argumentos la lista de proxies que devuelte get_proxies() y el número dentro de la lista. 

In [44]:
def set_proxi_for_req(proxies,n):
    proxi_http='http://' + str(proxies[n]['ip'])+ ':' +proxies[n]['port']
    proxi_https='https://' + str(proxies[n]['ip'])+ ':' +proxies[n]['port']
    proxi1 = {'http': str(proxi_http), "https": str(proxi_https)}
    return proxi1




proxies=get_proxies()

set_proxi_for_req(proxies, 5)

{'http': 'http://200.255.122.174:8080',
 'https': 'https://200.255.122.174:8080'}

Ese es el formato que requiere el paquete requests para fijar las proxies y se introduce la siguiente manera. 

In [45]:
# requests.get('url', proxies= 'proxi en formato visto')

Nosotras lo vamos a utilizar asi

In [46]:
#requests.get('url', proxies= set_proxi_for_req(proxies,16))

## Problemática de cambiar el proxy

Resulta que se las saben todas... cuando cambias el proxy muchas veces o algo así te aparece un capcha de "I'm not a robot" y eso jode que flipas, sobre todo cuando no eres el dios de la programación. He visto por hay que hay peña que es capaz de resolver capchas con python pero nunca con request... Por eso he decidido olvidarme de cambiar proxies y movidas. 

## Plan C: User Agents

Resulta que uno de los metodos para evitar ser baneado en páginas webs y esas cosas es cambiar el Agent-user es decir, si ven que hay muchas peticiones provenientes del mismo usuario pues te joden y te banean. 

En principio parece que funciona bien, tampoco quiero adelantar acontecimientos... Pero de momento voy plasmando el código que he hecho para obtener una lista de mogollón de Agent users. 



In [47]:
#DESCARGAR PEDAZO DE LISTA DE USER AGENTS        
def get_user_agents():
    user_agents= requests.get('http://www.useragentstring.com/pages/useragentstring.php?typ=Browser')
    user_soup = BeautifulSoup(user_agents.content, 'html.parser')
    user_soup1=user_soup.find_all('ul')
    
    users_list=[]
    for i in np.arange(0,len(user_soup1)):
        user_soup2= user_soup1[i].find_all('a')
        for j in np.arange(0,len(user_soup2)):
            users_list.append(user_soup2[j].text)
            
    return users_list



#PONER USER AGENTS EN FORMATO PARA HEADERS
def set_user_agents(users_list, n):
    headers = {'User-Agent': str(users_list[n])}
    return headers




users_list= get_user_agents()

set_user_agents(users_list, 1)


{'User-Agent': 'Mozilla/5.0 (compatible; U; ABrowse 0.6;  Syllable) AppleWebKit/420+ (KHTML, like Gecko)'}


Tambien hago una funcion para poner en el formato que a requests le gusta. La request se haría de la siguiente manera


In [48]:
#requests.get('url',headers='aqui el user agent en formato')

#requests.get('url',headers=set_user_agents(users_list,random.randrange(0,len(users_list))))

## Scraping como tal

Pos na, en lo que me como la cabeza pa poder hacer un chorro de request sin el problema de ser baneado paso a poner el código que uso para el scrap de la web como tal.


A continuación muestro el scrap de la página principal, principalmente porque tiene un detallito diferente al resto de páginas, pero tambien para obtener el númeor de páginas totales.

In [49]:

users_list= get_user_agents()        


first_req= requests.get('https://www.electrobuzz.net/',
                        headers=set_user_agents(users_list,
                                                 random.randrange(0, 
                                                                  len(users_list))))


soup = BeautifulSoup(first_req.content, 'html.parser')
soup1=soup.find_all('div', 
                    {'class': 'listing listing-blog listing-blog-1 clearfix columns-1 columns-1'})
soup2=soup1[0].find_all('article')


##vector de pags
total_pags_vec= get_vector_pags(soup)

#Info de la página principal
info_pag_1= obtn_link_title_img(soup2)


NameError: name 'get_vector_pags' is not defined


Para la obtencion de la info de la primera pagina se usa la misma funcion que para el scrap del resto de páginas. Esta funcion crea un lista por album con 3 cosas: LINK, NOMBRE E IMAGEN. notese que la imagen se guarda por defecto en path que yo he especificado a mano, pero seria interesante que este path se seteara de una manera más guay... nosecomolaverdad. 

In [ ]:
def obtn_link_title_img(bs4_element):
    list_link_title= []
    for i in np.arange(0,len(soup2)):
        title= soup2[i].find('a')['title']
        link= soup2[i].find('a')['href']
        
        
        
        link_img= soup2[0].find('a')['data-src']
        ext=link_img[-3:]
        filename_0=''.join(e for e in title if e.isalnum())
        filename= filename_0 + '.' + ext
        path='C:/Users/Oscar/Desktop/img/'
        file_ext= path + filename
        
        r = requests.get(link_img, stream=True)
        
        
        if r.status_code == 200:
            with open(file_ext, 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)
                
        if os.path.isfile(file_ext):
            link_title= list([link,title,file_ext])
            list_link_title.append(link_title)
        else: 
            link_title= list([link, title,str('NO')])
            list_link_title.append(link_title)
            
    return list_link_title
    


A este código hay que sumarle la siguiente funcion para realizar un scrap total. 


A continuacion el funcion que se emplearía en caso de que la idea de los proxis funcionará aunque parece que no va a ser asi. 

In [ ]:
def srch_info(vector_paginas, proxi1):
    
    url_base= 'https://www.electrobuzz.net/page'
        
    list_info=[]
    for i in vector_paginas:
        url_page= url_base + '/' + str(i)+ '/'
        
        req= requests.get(url_page, proxies= proxi1)
        soup = BeautifulSoup(req.content, 'html.parser')
        soup2=soup.find_all('article')
    
        
        list_info.append(obtn_link_title_img(soup2))
        
    
    return list_info



Y aquí la función que espero que sea definitiva, a esta funcion se le mete como argumentos el vector páginas, que es sin más un vector numérico desde 2 hasta la ultima página y ua que es la lista que hemos citado anteriormente de User-Agents. 

es guay porque yo con el vector páginas puedo elegir hasta donde o desde donde empiezo a descargar... incluso puedo meter páginas específicas aunque eso no tiene mucho sentido pero poder puedo. 

In [ ]:
def srch_info_chg_ua(vector_paginas, ua):
    
    url_base= 'https://www.electrobuzz.net/page'
        
    list_info=[]
    for i in vector_paginas:
        url_page= url_base + '/' + str(i)+ '/'
        headers=set_user_agents(ua,random.randrange(0,len(ua)))
        
        req= requests.get(url_page, headers= headers)
        soup = BeautifulSoup(req.content, 'html.parser')
        soup2=soup.find_all('article')
    
        
        list_info.append(obtn_link_title_img(soup2))
        
    
    return list_info
